In [1]:
import json
import spacy
from pathlib import Path

# Lade spaCy Modell (oder ersatzweise nlp = spacy.blank("de"))
nlp = spacy.load("de_core_news_md")

# Eingabe- und Ausgabe-Dateien
input_path = Path("ground_truth.json")
output_path = Path("ground_truth_cleaned.json")

def clean_labels(entry, nlp):
    text = entry["text"]
    labels = entry.get("labels", [])
    doc = nlp(text)
    cleaned_labels = []

    for label in labels:
        span = doc.char_span(label["start"], label["end"], label=label["label"], alignment_mode="expand")
        if span:
            cleaned_labels.append({
                "start": span.start_char,
                "end": span.end_char,
                "label": span.label_
            })
        else:
            print(f"⚠️ Label nicht ausrichtbar in {entry.get('file', '??')}: '{text[label['start']:label['end']]}' @ {label['start']}-{label['end']}")

    return {
        "file": entry.get("file", ""),
        "text": text,
        "labels": cleaned_labels
    }

def process_file(input_path, output_path):
    with input_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    cleaned_data = [clean_labels(entry, nlp) for entry in data]

    with output_path.open("w", encoding="utf-8") as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Bereinigte Datei gespeichert unter: {output_path.resolve()}")

# Ausführen
process_file(input_path, output_path)


✅ Bereinigte Datei gespeichert unter: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/Desktop/Veranstaltungen/Data Analytics in Applications/daia-eon/data/original/ground_truth_cleaned.json


In [11]:
import json
import spacy
from pathlib import Path

# Lade spaCy Modell
nlp = spacy.load("de_core_news_md")

# Pfade
input_path = Path("../../data/synthetic/synthetic_mails_option_b.json")
output_path = Path("../../data/synthetic/synthetic_mails_option_b_cleaned.json")


def clean_labels(entry, nlp):
    text = entry["text"]
    labels = entry.get("labels", [])
    doc = nlp(text)
    cleaned_labels = []

    for label in labels:
        start, end, label_name = label["start"], label["end"], label["label"]
        span_exact = doc.char_span(start, end, label=label_name, alignment_mode="strict")
        span_expand = doc.char_span(start, end, label=label_name, alignment_mode="expand")

        if span_exact:
            # Exakt zuweisbar
            cleaned_labels.append({
                "start": span_exact.start_char,
                "end": span_exact.end_char,
                "label": span_exact.label_
            })
        elif span_expand:
            # Zeige was expand erfassen würde
            print(f"🔍 Nicht exakt ausrichtbar ({entry.get('file', '??')}): '{text[start:end]}' @ {start}-{end}")
            print(f"➡️  Würde erweitert auf: '{span_expand.text}' @ {span_expand.start_char}-{span_expand.end_char}")
            print(f"    Tokens: {[t.text for t in span_expand]}")
            print("    ❓ Akzeptieren oder anpassen?\n")

            # Du kannst hier entscheiden, ob du trotzdem übernehmen willst
            # Zum Beispiel temporär automatisch übernehmen:
            cleaned_labels.append({
                "start": span_expand.start_char,
                "end": span_expand.end_char,
                "label": span_expand.label_
            })
        else:
            print(f"⚠️ Gar nicht zuweisbar ({entry.get('file', '??')}): '{text[start:end]}' @ {start}-{end}")

    return {
        "file": entry.get("file", ""),
        "text": text,
        "labels": cleaned_labels
    }

def process_file(input_path, output_path):
    with input_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    cleaned_data = [clean_labels(entry, nlp) for entry in data]

    with output_path.open("w", encoding="utf-8") as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Bereinigte Datei gespeichert unter: {output_path.resolve()}")

# Start
process_file(input_path, output_path)

🔍 Nicht exakt ausrichtbar (13): '404107498486' @ 53-65
➡️  Würde erweitert auf: '404107498486.' @ 53-66
    Tokens: ['404107498486.']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (16): 'Dipl.-Ing' @ 121-130
➡️  Würde erweitert auf: 'ADipl.-Ingesse' @ 120-134
    Tokens: ['ADipl.-Ingesse']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (16): 'Dr' @ 301-303
➡️  Würde erweitert auf: 'Dr.' @ 301-304
    Tokens: ['Dr.']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (18): 'Aurelia' @ 340-347
➡️  Würde erweitert auf: 'Aurelia_Löffler' @ 340-355
    Tokens: ['Aurelia_Löffler']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (18): 'Löffler' @ 348-355
➡️  Würde erweitert auf: 'Aurelia_Löffler' @ 340-355
    Tokens: ['Aurelia_Löffler']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausrichtbar (21): '87-86' @ 190-195
➡️  Würde erweitert auf: '087-86' @ 189-195
    Tokens: ['087', '-', '86']
    ❓ Akzeptieren oder anpassen?

🔍 Nicht exakt ausr

KeyboardInterrupt: 

In [14]:
import json
import spacy
from pathlib import Path

# Lade SpaCy-Modell
nlp = spacy.load("de_core_news_md")

# Pfade
input_path   = Path("../../data/synthetic/synthetic_mails_option_b.json")
cleaned_path = Path("../../data/synthetic/synthetic_mails_option_b_cleaned_new.json")
failed_path  = Path("../../data/synthetic/synthetic_mails_option_b_failed_new.json")

def try_expand_one_char(doc, text, start, end, label_name):
    # Ein Zeichen links
    if start > 0:
        span = doc.char_span(start-1, end, label=label_name, alignment_mode="strict")
        if span:
            return span
    # Ein Zeichen rechts
    if end < len(text):
        span = doc.char_span(start, end+1, label=label_name, alignment_mode="strict")
        if span:
            return span
    return None

def clean_labels(entry, nlp):
    """
    Gibt zurück:
      - cleaned_labels: Liste der erfolgreich bereinigten spans
      - failures: Liste von Failure-Records mit orig_label + suggestion
    """
    text   = entry.get("text", "")
    labels = entry.get("labels", [])
    doc    = nlp(text)

    cleaned_labels = []
    failures = []

    for label in labels:
        start, end, label_name = label["start"], label["end"], label["label"]

        # 1) strict
        span = doc.char_span(start, end, label=label_name, alignment_mode="strict")
        if span:
            cleaned_labels.append({
                "start": span.start_char,
                "end":   span.end_char,
                "label": span.label_
            })
            continue

        # 2) single-char expand
        span = try_expand_one_char(doc, text, start, end, label_name)
        if span:
            cleaned_labels.append({
                "start": span.start_char,
                "end":   span.end_char,
                "label": span.label_
            })
            continue

        # 3) token-extend fallback: nur vorschlagen
        span_fb = doc.char_span(start, end, label=label_name, alignment_mode="expand")
        if span_fb:
            suggested = {
                "start": span_fb.start_char,
                "end":   span_fb.end_char,
                "text":  span_fb.text
            }
        else:
            suggested = None

        # 4) record failure
        failures.append({
            "orig_label": {
                "start": start,
                "end":   end,
                "label": label_name,
                "text":  text[start:end]
            },
            "suggestion": suggested
        })

    return cleaned_labels, failures

def process_file(input_path, cleaned_path, failed_path):
    # 1) Einlesen
    with input_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    cleaned_data = []
    failed_data  = []

    # 2) pro Eintrag:
    for entry in data:
        cleaned_labels, failures = clean_labels(entry, nlp)

        if failures:
            # Mindestens ein Label ist fehlgeschlagen →
            # wir speichern die ganze Mail in failed_data:
            failed_data.append({
                "file":       entry.get("file", ""),
                "text":       entry.get("text", ""),
                "orig_labels": entry.get("labels", []),
                "failures":   failures
            })
        else:
            # Alle Labels sauber ⇒ in cleaned_data
            cleaned_data.append({
                "file":   entry.get("file", ""),
                "text":   entry.get("text", ""),
                "labels": cleaned_labels
            })

    # 3) Speichern
    with cleaned_path.open("w", encoding="utf-8") as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=2)
    with failed_path.open("w", encoding="utf-8") as f:
        json.dump(failed_data, f, ensure_ascii=False, indent=2)

    # 4) Reporting auf E-Mail-Ebene
    total_emails       = len(data)
    total_cleaned      = len(cleaned_data)
    total_failed_mail  = len(failed_data)

    print(f"📄 Gesamte Mails verarbeitet: {total_emails}")
    print(f"✅ Vollständig bereinigt:     {total_cleaned}")
    print(f"⚠️  Mit Fehlern (failed):      {total_failed_mail}\n")

    if total_failed_mail:
        print("Beispiele fehlerhafter Mails:")
        for rec in failed_data[:5]:
            print(f" • Datei {rec['file']!r}:")
            for f in rec["failures"]:
                orig = f["orig_label"]
                print(f"    – Orig: '{orig['text']}' @{orig['start']}-{orig['end']} ({orig['label']})")
                if f["suggestion"]:
                    s = f["suggestion"]
                    print(f"      ↳ Vorschlag: '{s['text']}' @{s['start']}-{s['end']}")
                else:
                    print("      ↳ Kein Vorschlag möglich")
            print()

    print(f"✅ Bereinigt-Datei: {cleaned_path.resolve()}")
    print(f"✅ Failed- Datei:   {failed_path.resolve()}")

# Ausführen
process_file(input_path, cleaned_path, failed_path)

📄 Gesamte Mails verarbeitet: 14360
✅ Vollständig bereinigt:     13506
⚠️  Mit Fehlern (failed):      854

Beispiele fehlerhafter Mails:
 • Datei '16':
    – Orig: 'Dipl.-Ing' @121-130 (TITEL)
      ↳ Vorschlag: 'ADipl.-Ingesse' @120-134

 • Datei '18':
    – Orig: 'Aurelia' @340-347 (VORNAME)
      ↳ Vorschlag: 'Aurelia_Löffler' @340-355
    – Orig: 'Löffler' @348-355 (NACHNAME)
      ↳ Vorschlag: 'Aurelia_Löffler' @340-355

 • Datei '49':
    – Orig: 'Eugen' @205-210 (VORNAME)
      ↳ Vorschlag: 'EugenKlemt' @205-215
    – Orig: 'Klemt' @210-215 (NACHNAME)
      ↳ Vorschlag: 'EugenKlemt' @205-215

 • Datei '76':
    – Orig: 'Mirjam' @184-190 (VORNAME)
      ↳ Vorschlag: 'MirjamDowerg' @184-196
    – Orig: 'Dowerg' @190-196 (NACHNAME)
      ↳ Vorschlag: 'MirjamDowerg' @184-196

 • Datei '79':
    – Orig: 'Klapp' @189-194 (NACHNAME)
      ↳ Vorschlag: 'Klapp402' @189-197
    – Orig: '402 862 472 874' @194-209 (VERTRAGSNUMMER)
      ↳ Vorschlag: 'Klapp402 862 472 874' @189-209

✅ Bereini